In [1]:
import os
import tensorflow as tf
tf.enable_eager_execution()
import pandas as pd
import math
import numpy as np
import itertools
import collections
from datetime import datetime

In [2]:
from waymo_open_dataset.utils import range_image_utils
from waymo_open_dataset.utils import transform_utils
from waymo_open_dataset.utils import  frame_utils
from waymo_open_dataset import dataset_pb2 as open_dataset

In [3]:
# get all segment file names and save in Filenames
def scan(folder):        
    x=[]
    for path,dirs,files in os.walk(folder):
        for f in files:
            if f.startswith('segment'):
                x.append(os.path.join(path,f))
    return(x)

folder = './waymo-od/data'

Filenames=scan(folder)

In [4]:
def getID(f):         #get the ID of segment from file name
    f = f.replace('_with_camera_labels.tfrecord', '')
    f = f.split('segment-',1)[1]
    return f

In [5]:
### Extract segment files in Frames, each file consists of multiple frames

Frames = {}

for file in Filenames[0:3]:
    SegID = getID(file)
    Segment = []
    dataset = tf.data.TFRecordDataset(file, compression_type='')
    for data in dataset:
        frame = open_dataset.Frame()
        frame.ParseFromString(bytearray(data.numpy()))
        Segment.append(frame)
    Frames[SegID] = Segment

In [ ]:
np.matmul(
          np.reshape(np.array(Frames['10231929575853664160_1160_000_1180_000'][2]
                              .pose.transform),[4,4]),(0,0,0,1))[0]

In [ ]:
np.matmul(
          np.reshape(np.array(Frames['10231929575853664160_1160_000_1180_000'][1]
                              .pose.transform),[4,4]),(0,0,0,1))[0]

In [ ]:
np.matmul(
          np.reshape(np.array(Frames['10231929575853664160_1160_000_1180_000'][0]
                              .pose.transform),[4,4]),(0,0,0,1))[0]

In [ ]:
np.array(Frames['10231929575853664160_1160_000_1180_000'][0].pose.transform).reshape(4,4)

## Each frame includes:
#### 1. ***context***: similar for all the frames in one segment
#### 2. ***timestamp_micros*** : should be devided to 1000000
#### 3. ***pose***
#### 4. ***laser_labels*** : includes
#### 5. ***camera_labels***

In [6]:
def CountLLabels(frame):    #number of laser labels in a frame
    return len(frame.laser_labels)

def converttime(t):
    t = t/1000000
    t = datetime.fromtimestamp(t)
    return t

In [34]:
def Ped2df(SegID, tstamp, pedLoc, 
           pedMeta,VehPos,tday,w):
    
    length = len(pedLoc)
    t,px, py, pz,\
    px_GF,py_GF,pz_GF,\
    vx_GF,vy_GF,vz_GF,\
    pw, pl, pht,\
    phd, psx, psy,\
    paccx, paccy = ([] for i in range(18))
    
    for i in range(length):
        t.append(tstamp[i])
        pw.append(pedLoc[i].width)       
        pl.append(pedLoc[i].length)
        pht.append(pedLoc[i].height)
        phd.append(pedLoc[i].heading)
        psx.append(pedMeta[i].speed_x)
        psy.append(pedMeta[i].speed_y)
        paccx.append(pedMeta[i].accel_x)
        paccy.append(pedMeta[i].accel_y)

        # Ped position in Vehicle Frame
        PedPoseVF = np.array([pedLoc[i].center_x,
                              pedLoc[i].center_y,
                              pedLoc[i].center_z,1])
        px.append(PedPoseVF[0])
        py.append(PedPoseVF[1])
        pz.append(PedPoseVF[2])
        # Ped position in Global Frame
        PedPoseGF = np.matmul(np.reshape(np.array(VehPos[i].transform),[4,4]),PedPoseVF)
        px_GF.append(PedPoseGF[0])
        py_GF.append(PedPoseGF[1])
        pz_GF.append(PedPoseGF[2])
        # SDC position in Global Frame
        VehPoseGF = np.matmul(np.reshape(np.array(VehPos[i].transform),[4,4]),[0,0,0,1])
        vx_GF.append(VehPoseGF[0])
        vy_GF.append(VehPoseGF[1])
        vz_GF.append(VehPoseGF[2])
    
    data = np.array([t, px, py, pz,
                     px_GF, py_GF, pz_GF,
                     vx_GF, vy_GF, vz_GF,
                     pw, pl, pht, phd,
                     psx, psy, paccx, paccy]).T.tolist()
    
    columnsnames= ['Timestamp', 'x', 'y', 'z',
                   'x_GF', 'y_GF', 'pz_GF',
                    'x_GF', 'y_GF', 'z_GF',
                   'Width', 'Length','Height','Heading',
                   'Speed x','Speed y', 'accel x', 'accel y']
    pedDF = pd.DataFrame(data, columns = columnsnames)
    
    #transform = pedDF['vpose'].apply(pd.Series)
    #transform = transform.rename(columns= lambda x: 'AVtransform_' +str(x))
    #pedDF = pedDF.drop(columns= 'vpose')
    #pedDF = pd.concat([pedDF[:],transform[:]],axis=1)
    
    pedDF['Weather'] = w
    pedDF['Time of Day'] = tday
    pedDF['Segment ID'] = SegID
    return pedDF
 

In [35]:
def PedExtract(SegID):
    pedLoc = collections.defaultdict(list)                             #pedestrians label box
    pedMeta = collections.defaultdict(list)                            #pedestrians label metadata
    VehPos =  collections.defaultdict(list)                            #SDC transformation matrix
    tstamp = collections.defaultdict(list) 
    tday = Frames[SegID][0].context.stats.time_of_day
    w = Frames[SegID][0].context.stats.weather
    
    for j in range(len(Frames[SegID])):
        t = Frames[SegID][j].timestamp_micros                      #timestamp
        vpos = Frames[SegID][j].pose                                
        Nlabels = CountLLabels(Frames[SegID][j])                   # number of laser labels
       
        for k in range(Nlabels):
            LidarLab = Frames[SegID][j].laser_labels[k]
            if  LidarLab.type == 2:                      # type 2: pedestrian
                
                PedID = LidarLab.id

                tstamp[PedID].append(t)
                pedLoc[PedID].append(LidarLab.box)

                
                pedMeta[PedID].append(LidarLab.metadata)
                
                
                VehPos[PedID].append(Frames[SegID][j].pose)
    
    Ped = {}
    for p in list(pedLoc.keys()):
        Ped[p] = Ped2df(SegID, tstamp[p], pedLoc[p],
                        pedMeta[p],VehPos[p],tday, w)
        
    
    return Ped      

In [36]:
def MultiFrame(Frames):
    PedTraj = {}
    for i in range(len(Frames)):
        SegID = list(Frames.keys())[i]
        PedTraj.update(PedExtract(SegID))
    return PedTraj
    

In [37]:
PedData = MultiFrame(Frames)

In [43]:
PedIDs = list(PedData.keys())

In [45]:
PedData[PedIDs[0]]
    
    

,Timestamp,x,y,z,x_GF,y_GF,pz_GF,x_GF,y_GF,z_GF,...,Length,Height,Heading,Speed x,Speed y,accel x,accel y,Weather,Time of Day,Segment ID
0,1.557844e+15,-22.254435,9.129411,0.839429,-1553.806695,14431.570786,-10.592450,-1548.499013,14455.047079,-10.539,...,1.08548,1.74,-0.198821,0.046410,1.439603,0.002425,0.075235,sunny,Day,10231929575853664160_1160_000_1180_000
1,1.557844e+15,-22.867838,9.102094,0.903977,-1553.802125,14431.712557,-10.580454,-1548.619559,14455.790331,-10.509,...,1.08548,1.74,-0.198812,0.046652,1.447127,0.002425,0.075235,sunny,Day,10231929575853664160_1160_000_1180_000
2,1.557844e+15,-23.491898,9.053602,0.981302,-1553.797553,14431.854383,-10.568454,-1548.750023,14456.538684,-10.481,...,1.08548,1.74,-0.198121,0.046895,1.454650,0.002425,0.075235,sunny,Day,10231929575853664160_1160_000_1180_000
3,1.557844e+15,-24.122108,9.011221,1.027904,-1553.792981,14431.996194,-10.556454,-1548.875288,14457.293221,-10.449,...,1.08548,1.74,-0.197538,0.047137,1.462174,0.002425,0.075235,sunny,Day,10231929575853664160_1160_000_1180_000
4,1.557844e+15,-24.755325,8.985655,1.027816,-1553.788409,14432.138019,-10.544454,-1549.002205,14458.055195,-10.415,...,1.08548,1.74,-0.197535,0.047380,1.469698,0.002425,0.075235,sunny,Day,10231929575853664160_1160_000_1180_000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,1.557844e+15,-74.653427,8.296791,-2.715913,-1555.946964,14445.055643,-9.989132,-1559.146117,14520.125269,-8.075,...,1.08548,1.74,0.008280,-0.223543,1.460792,0.047397,-0.084316,sunny,Day,10231929575853664160_1160_000_1180_000
90,1.557844e+15,-75.208184,8.279263,-2.651754,-1555.968204,14445.199219,-9.964681,-1559.252248,14520.813550,-8.084,...,1.08548,1.74,0.006053,-0.219005,1.453354,0.044502,-0.072941,sunny,Day,10231929575853664160_1160_000_1180_000
91,1.557844e+15,-75.755509,8.272397,-2.671016,-1555.989438,14445.342749,-9.940238,-1559.358710,14521.498460,-8.091,...,1.08548,1.74,0.003637,-0.214611,1.446508,0.040430,-0.059422,sunny,Day,10231929575853664160_1160_000_1180_000
92,1.557844e+15,-76.297263,8.282611,-2.735257,-1556.010680,14445.486336,-9.915785,-1559.462327,14522.181406,-8.097,...,1.08548,1.74,0.001124,-0.210487,1.440448,0.035531,-0.044920,sunny,Day,10231929575853664160_1160_000_1180_000


In [33]:
t,px,
py,pz,px_GF,py_GF,pz_GF,vx_GF,vy_GF,vz_GF,pw, pl, pht,phd, psx, psy,paccx, paccy = ([] for i in range(18))

ValueError: too many values to unpack (expected 16)